In [1]:
# Tensorflow to open-source’owy framework stworzony przez Google’a do obliczeń numerycznych. 
# Oferuje on zestaw narzędzi służących do projektowania, trenowania oraz douczania (ang. fine-tuning) sieci neuronowych.
# TensorBoard to narzędzie służące do wizualizacji danych. Pozwala prezentować w czytelny sposób m.in. grafy obliczeniowe 
# sieci neuronowych (ang. computational graph), dane wejściowe, dane wyjściowe czy postęp treningu. Dzięki temu ułatwia 
# ono znajdowanie błędów w architekturze oraz optymalizację sieci neuronowych.

In [2]:
# Software requirements
# NVIDIA® GPU drivers —CUDA 10.1 requires 418.x or higher.
# CUDA® Toolkit —TensorFlow supports CUDA 10.1 (TensorFlow >= 2.1.0)
# CUPTI ships with the CUDA Toolkit.
# cuDNN SDK (>= 7.6)
#(Optional) TensorRT 6.0 to improve latency and throughput for inference on some models.

In [3]:
# Source: Python for Data Science and Machine Learning Bootcamp, kurs z platformy Udemy, Stworzony przez Jose Portilla
# Ostatnia aktualizacja: 5/2020

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3

In [2]:
os.chdir(r"C:\Users\Person\Desktop\WSB\Analysis-and-forecast-of-stock-exchange-price-based-on-selected-indicators-using-machine-learning")

In [3]:
conn = sqlite3.connect('data\predictions.db')
c = conn.cursor()
indicator="SP500"
# SELECT QUERY dla indexu SP500
df = pd.read_sql_query("""select x."Date", co."Index" as Crude_Oil, 
cr."Index" as Copper, s."Index" as Silver, p."Index" as Platinum,
ip."Index" as Industrial_Production, pa."Index" as Palladium, x."Index" as """+indicator+"""
 from """+indicator+""" as x
  LEFT OUTER JOIN Crude_Oil as co ON x.Date=co.Date
  LEFT OUTER JOIN Copper as cr ON x.Date=cr.Date
  LEFT OUTER JOIN Silver as s ON x.Date=s.Date
  LEFT OUTER JOIN Platinum as p ON x.Date=p.Date
  LEFT OUTER JOIN Industrial_Prod as ip ON strftime('%Y-%m', x.Date)=strftime('%Y-%m', ip.Date)
  LEFT OUTER JOIN Palladium as pa ON x.Date=pa.Date
Where x."Index" and co."Index" and cr."Index" and s."Index" and p."Index"
and ip."Index" and pa."Index" IS NOT NULL
and x.Date < '2020-04-01'
""", conn)

In [4]:
df

,Date,Crude_Oil,Copper,Silver,Platinum,Industrial_Production,Palladium,SP500
0,2010-01-04 00:00:00,81.52,3.4060,17.170,1523.90,91.685,421.40,1132.99
1,2010-01-06 00:00:00,83.12,3.4945,17.890,1558.40,91.685,427.20,1137.14
2,2010-01-07 00:00:00,82.60,3.4270,18.090,1559.40,91.685,424.55,1141.69
3,2010-01-08 00:00:00,82.74,3.4005,18.120,1570.60,91.685,425.15,1144.98
4,2010-01-11 00:00:00,82.54,3.4410,18.840,1592.50,91.685,431.95,1146.98
...,...,...,...,...,...,...,...,...
2510,2020-03-25 00:00:00,24.49,2.2040,13.965,745.50,103.664,2247.60,2475.56
2511,2020-03-26 00:00:00,22.60,2.1780,14.415,737.10,103.664,2226.10,2630.07
2512,2020-03-27 00:00:00,21.51,2.1720,14.315,740.82,103.664,2196.80,2541.47
2513,2020-03-30 00:00:00,20.09,2.1555,14.055,723.84,103.664,2197.60,2626.65


In [5]:
# Ze względu na to że dane trainowe mają mieć wartości float, usuwam kolumne daty
df = df.drop('Date',axis=1)

In [6]:
df

,Crude_Oil,Copper,Silver,Platinum,Industrial_Production,Palladium,SP500
0,81.52,3.4060,17.170,1523.90,91.685,421.40,1132.99
1,83.12,3.4945,17.890,1558.40,91.685,427.20,1137.14
2,82.60,3.4270,18.090,1559.40,91.685,424.55,1141.69
3,82.74,3.4005,18.120,1570.60,91.685,425.15,1144.98
4,82.54,3.4410,18.840,1592.50,91.685,431.95,1146.98
...,...,...,...,...,...,...,...
2510,24.49,2.2040,13.965,745.50,103.664,2247.60,2475.56
2511,22.60,2.1780,14.415,737.10,103.664,2226.10,2630.07
2512,21.51,2.1720,14.315,740.82,103.664,2196.80,2541.47
2513,20.09,2.1555,14.055,723.84,103.664,2197.60,2626.65


In [7]:
# Training And Test Data
X = df.drop('SP500',axis=1).values
y = df['SP500'].values

In [8]:
y

array([1132.99, 1137.14, 1141.69, ..., 2541.47, 2626.65, 2584.59])

In [9]:
X

array([[8.15200e+01, 3.40600e+00, 1.71700e+01, 1.52390e+03, 9.16850e+01,
        4.21400e+02],
       [8.31200e+01, 3.49450e+00, 1.78900e+01, 1.55840e+03, 9.16850e+01,
        4.27200e+02],
       [8.26000e+01, 3.42700e+00, 1.80900e+01, 1.55940e+03, 9.16850e+01,
        4.24550e+02],
       ...,
       [2.15100e+01, 2.17200e+00, 1.43150e+01, 7.40820e+02, 1.03664e+02,
        2.19680e+03],
       [2.00900e+01, 2.15550e+00, 1.40550e+01, 7.23840e+02, 1.03664e+02,
        2.19760e+03],
       [2.04800e+01, 2.22800e+00, 1.39300e+01, 7.28800e+02, 1.03664e+02,
        2.30480e+03]])

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# Dzielenie tablic na losowe podzbiory trainowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=101)

In [12]:
# Scalling data
from sklearn.preprocessing import MinMaxScaler

In [13]:
scaler = MinMaxScaler()

In [14]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [15]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
# Creating model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

C:\Users\Person\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Person\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Person\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Person\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [17]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [18]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [19]:
from datetime import datetime

In [20]:
datetime.now().strftime("%Y-%m-%d--%H%M")

'2020-05-13--1308'

In [21]:
# WINDOWS: Use "logs\\fit"
# MACOS/LINUX: Use "logs\fit"

log_directory = 'logs\\fit'

board = TensorBoard(log_dir=log_directory,histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch',
    profile_batch=2,
    embeddings_freq=1)

In [22]:
# Teraz utwórzę warstwy modelu
model = Sequential()
model.add(Dense(units=30,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=15,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [23]:
# Train the model
model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop,board]
          )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1886 samples, validate on 629 samples
Epoch 1/600
1088/1886 [================>.............] - ETA: 0s - loss: -1095.0595

ValueError: To visualize embeddings, embeddings_data must be provided.

In [24]:
print(log_directory)

logs\fit


In [26]:
pwd

'C:\\Users\\Person\\Desktop\\WSB\\Analysis-and-forecast-of-stock-exchange-price-based-on-selected-indicators-using-machine-learning'

### Użyj cd w wierszu poleceń, aby zmienić katalog na ścieżkę pliku zgłoszoną przez pwd lub bieżącą lokalizację pliku .py
### Następnie uruchom ten kod w wierszu polecenia lub terminalu, wpisując - 'tensorboard --logdir logs\fit'
### Tensorboard będzie działał lokalnie w twojej przeglądarce pod adresem [http://localhost:6006/](http://localhost:6006/)


In [25]:
# Możemy użyć interfejsów API tensorboard.notebook, aby uzyskać nieco większą kontrolę:
from tensorboard import notebook
notebook.list()

No known TensorBoard instances running.
